In [9]:
import sys
import pandas as pd
import numpy as np
import hdbscan

In [10]:
# Load up code to run ICIM
# Available from : https://github.com/felixhorns/FlyPN
sys.path.append("../code/ICIM")
import sct
reload(sct)
# Used because of nature of ICIM library
pd.options.mode.chained_assignment = None  # default='warn'

In [11]:
# The input for ICIM is log2 transformed data in practice
# See manuscript, github
df = pd.read_csv("../data/02_filtered_kenyon_cells/CLEAN_LOG2TRANSFORM_kenyon_cells.csv")
df = df.set_index("symbol")

In [12]:
# Load up metadata associated with these cells
metadata = pd.read_csv("../data/02_filtered_kenyon_cells/metadata_kenyon_cells.csv")
metadata = metadata.set_index("CellID")

In [13]:
metadata.head()

,Age,Gender,Genotype,Replicate,nGene,nUMI,cell_type_id,is_kc
CellID,,,,,,,,
ACATACGAGGGCTTCC-DGRP-551_0d_r1,0,Female,DGRP-551,DGRP-551_0d_Rep1,1328,3340.0,8.0,1
ACCCACTTCACTCTTA-DGRP-551_0d_r1,0,Female,DGRP-551,DGRP-551_0d_Rep1,1613,4580.0,8.0,1
ACCGTAAAGATAGTCA-DGRP-551_0d_r1,0,Male,DGRP-551,DGRP-551_0d_Rep1,1466,4349.0,22.0,1
ACTTACTAGTGGTAAT-DGRP-551_0d_r1,0,Male,DGRP-551,DGRP-551_0d_Rep1,1174,2942.0,8.0,1
ACTTGTTCATGGTTGT-DGRP-551_0d_r1,0,Male,DGRP-551,DGRP-551_0d_Rep1,1410,3620.0,8.0,1


In [14]:
# ICIM takes a filtered dataset as one of its arguments
# Filtering requirements are taken from an up-to-date tutorial
# from the hemberg lab's tutorial on seurat, to mimic the presumed
# level of filtering used for the original seurat analsyis
# Source https://hemberg-lab.github.io/scRNA.seq.course/seurat-chapter.html#normalization

# filtered_df
f_df = df.copy(deep=True)

# seuset <- CreateSeuratObject(
#     raw.data = counts(deng),
#     min.cells = 3, 
#     min.genes = 200
# )

# Check to make sure each gene is present in at least 3 cells
def check_min_num_cells(row):
    num_pos = 0
    for gene_count in row:
        if gene_count > 0:
            num_pos += 1
            if num_pos >= 3:
                return True
    return False
   

min_num_cells = f_df.apply(check_min_num_cells, axis=1)
f_df = f_df[min_num_cells]

In [15]:
# Check to make sure each cell has at least 200 genes
def check_min_num_genes(col):
    num_pos = 0
    for gene_count in col:
        if gene_count > 0:
            num_pos += 1
            if num_pos >= 200:
                return True
    return False

min_num_genes = f_df.apply(check_min_num_genes, axis=0)
f_df = f_df.loc[:, min_num_genes]

In [16]:
f_df.shape

(8255, 2848)

In [17]:
# Load the ICIM analysis object with associated data
# See https://github.com/felixhorns/FlyPN/blob/master/analysis/GH146_Fig2.ipynb 
# for full featured example

reload(sct)
from scipy.cluster import hierarchy
myICIM = sct.ICIM(f_df, df, TFs=[], CSMs=[], exclude=[], N=100,
                  correlation_cutoff=0.5,
                  min_hits=3,
                  exclude_max=2,
                  dropout_rate_low=0.3,
                  dropout_rate_high=1.0,
                  metric="correlation",
                  stop_condition="linkage_dist",
                  N_stop=50,
                  linkage_dist_stop=0.2)

In [18]:
# Run step command to iterate over first run of ICIM algo
# QC / Status Check
myICIM.step("0", verbose=True)

Found 23 genes
Child populations 2845 3


['00']

In [ ]:
# Run full ICIM pipeline
myICIM.calc(verbose=True)

In [59]:
genes_KC_ICIM = myICIM.get_all_markers()
print "Genes found by ICIM", len(genes_KC_ICIM)

Genes found by ICIM 267


In [21]:
# with open("../data/03_ICIM_analysis/KC_genes_ICIM.txt", 'w') as out:
#     for x in genes_KC_ICIM:
#         out.write(x + "\n")

genes_KC_ICIM = []
with open("../data/03_ICIM_analysis/KC_genes_ICIM.txt", 'r') as infile:
    for line in infile:
        genes_KC_ICIM.append(line.strip())

In [22]:
# Subset data to genes desired
# Twice filtered df
f2_df = f_df.loc[genes_KC_ICIM]

# Calculate TSNE
reload(sct)
from sklearn.manifold import TSNE
myTSNE = sct.TSNE(f2_df, df, metadata)
myTSNE.calc_TSNE(perplexity=10, learning_rate=250, early_exaggeration=4.0, method="exact", random_state=1)

[t-SNE] Computed conditional probabilities for sample 1000 / 2848
[t-SNE] Computed conditional probabilities for sample 2000 / 2848
[t-SNE] Computed conditional probabilities for sample 2848 / 2848
[t-SNE] Mean sigma: 0.100808
[t-SNE] Iteration 50: error = 22.0726243, gradient norm = 0.0517549 (50 iterations in 14.230s)
[t-SNE] Iteration 100: error = 20.6186237, gradient norm = 0.0479926 (50 iterations in 14.080s)
[t-SNE] Iteration 150: error = 20.5975731, gradient norm = 0.0359180 (50 iterations in 14.772s)
[t-SNE] Iteration 200: error = 20.4600786, gradient norm = 0.0500425 (50 iterations in 17.499s)
[t-SNE] Iteration 250: error = 20.4023608, gradient norm = 0.0406528 (50 iterations in 15.931s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 20.402361
[t-SNE] Iteration 300: error = 2.6382573, gradient norm = 0.0023594 (50 iterations in 14.128s)
[t-SNE] Iteration 350: error = 2.4133272, gradient norm = 0.0005949 (50 iterations in 15.642s)
[t-SNE] Iteration 400: err

In [31]:
myTSNE.df_libs = myTSNE.df_libs.loc[myTSNE.df.columns,:]

In [32]:
# Using HDBSCAN to call clusters automatically, generate metadata for them
clusterer = hdbscan.HDBSCAN(min_cluster_size=30, min_samples=3).fit(myTSNE.X_tsne)
labels_HDBSCAN = clusterer.labels_

In [33]:
print "Clusters", max(labels_HDBSCAN)+1

Clusters 7


In [34]:
zero_axis = myTSNE.X_tsne[:,0]
one_axis = myTSNE.X_tsne[:,1]
myTSNE.df_libs['x'] = zero_axis
myTSNE.df_libs['y'] = one_axis
myTSNE.df_libs['hdb_clust'] = labels_HDBSCAN
myTSNE.df_libs.index.name = "CellID"

In [38]:
def gen_kc_type(row):
    if int(row.cell_type_id) == 8:
        return 'G-KC'
    elif int(row.cell_type_id) == 22:
        return 'a/b-KC'
    elif int(row.cell_type_id) == 28:
        return "a'/b'-KC"
    
myTSNE.df_libs['subtype'] = myTSNE.df_libs.apply(gen_kc_type, axis=1)

In [40]:
myTSNE.df_libs.head()

,Age,Gender,Genotype,Replicate,nGene,nUMI,cell_type_id,is_kc,x,y,hdb_clust,subtype
CellID,,,,,,,,,,,,
ACATACGAGGGCTTCC-DGRP-551_0d_r1,0,Female,DGRP-551,DGRP-551_0d_Rep1,1328,3340.0,8.0,1,-16.732498,55.850990,3,G-KC
ACCCACTTCACTCTTA-DGRP-551_0d_r1,0,Female,DGRP-551,DGRP-551_0d_Rep1,1613,4580.0,8.0,1,-19.377689,40.539936,3,G-KC
ACCGTAAAGATAGTCA-DGRP-551_0d_r1,0,Male,DGRP-551,DGRP-551_0d_Rep1,1466,4349.0,22.0,1,-1.656786,-67.122963,2,a/b-KC
ACTTACTAGTGGTAAT-DGRP-551_0d_r1,0,Male,DGRP-551,DGRP-551_0d_Rep1,1174,2942.0,8.0,1,-15.413997,46.957966,3,G-KC
ACTTGTTCATGGTTGT-DGRP-551_0d_r1,0,Male,DGRP-551,DGRP-551_0d_Rep1,1410,3620.0,8.0,1,-19.743160,40.827847,3,G-KC


In [41]:
myTSNE.df_libs.to_csv("../data/03_ICIM_analysis/KC_ICIM_TSNE_data.csv")

In [42]:
pp = pd.read_csv("../data/03_ICIM_analysis/KC_ICIM_TSNE_data.csv")

In [43]:
df_list = []
for var in ['Age', 'Gender', 'Genotype']:
    tdf = pp.groupby(['hdb_clust', var])['subtype'].count().to_frame(name='num_cells').reset_index().set_index('hdb_clust')
    tdf['variable'] = tdf[var]
    tdf['variable_cat'] = var
    tdf = tdf.drop(var, axis=1)
    df_list.append(tdf)

In [44]:
histo = pd.concat(df_list)
histo = histo[['variable_cat', 'variable', 'num_cells']]

In [45]:
total_cells = pp.groupby(['hdb_clust'])['subtype'].count().to_frame(name='total_number_cells_in_cluster')

In [46]:
histo = histo.join(total_cells)

In [47]:
histo['proportion'] = histo.num_cells / histo.total_number_cells_in_cluster

In [50]:
histo.head()

,variable_cat,variable,num_cells,total_number_cells_in_cluster,proportion
hdb_clust,,,,,
-1,Age,0,6,62,0.096774
-1,Age,1,15,62,0.241935
-1,Age,3,11,62,0.177419
-1,Age,6,11,62,0.177419
-1,Age,9,8,62,0.129032


In [51]:
histo.to_csv("../data/03_ICIM_analysis/KC_bias_data.csv")